Parse PDb files and extract 3D coordinates of atoms and Generation of Voronoi tessellation 

For tumor antigenic proteins and non antigenic protein

In [ ]:
import os
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
from Bio import PDB

###############################################################################
# 1. COLOR SCHEME & HELPER FUNCTIONS
###############################################################################

# Custom color scheme for the 20 amino acids
amino_acid_colors = {
    "ALA": "#FF0000",  # Alanine
    "ARG": "#00FF00",  # Arginine
    "ASN": "#0000FF",  # Asparagine
    "ASP": "#FF00FF",  # Aspartic acid
    "CYS": "#FFFF00",  # Cysteine
    "GLN": "#FF8000",  # Glutamine
    "GLU": "#8000FF",  # Glutamic acid
    "GLY": "#808080",  # Glycine
    "HIS": "#008000",  # Histidine
    "ILE": "#00FFFF",  # Isoleucine
    "LEU": "#0040FF",  # Leucine
    "LYS": "#800000",  # Lysine
    "MET": "#008080",  # Methionine
    "PHE": "#FF8080",  # Phenylalanine
    "PRO": "#800080",  # Proline
    "SER": "#40E0D0",  # Serine
    "THR": "#A52A2A",  # Threonine
    "TRP": "#FFD700",  # Tryptophan
    "TYR": "#DC143C",  # Tyrosine
    "VAL": "#006400"   # Valine
}

# Default color for unknown residues
default_color = "#000000"


def extract_residue_coordinates_and_types(pdb_file):
    """
    Extract residue-level coordinates and residue types from a PDB file.
    Uses the alpha carbon (Cα) of each residue to represent its position.
    """
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file)

    residues = []
    residue_types = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if "CA" in residue:  # Only consider Cα atoms
                    residues.append(residue["CA"].coord)
                    res_name = residue.get_resname().strip()
                    residue_types.append(amino_acid_colors.get(res_name, default_color))

    return np.array(residues), residue_types


def plot_voronoi_2d_residue_colored(residues, residue_types, pdb_name, output_folder):
    """
    Generates and saves a 2D Voronoi diagram where each region is colored
    based on the amino acid type.
    """
    os.makedirs(output_folder, exist_ok=True)

    # Project to 2D (XY plane only)
    points_2d = residues[:, :2]

    # If fewer than 2 points, Voronoi will fail
    if len(points_2d) < 2:
        print(f"Not enough points to generate Voronoi for {pdb_name}. Skipping.")
        return

    vor = Voronoi(points_2d)

    fig, ax = plt.subplots(figsize=(8, 8))

    # Color each Voronoi region
    for region_idx, residue_color in zip(vor.point_region, residue_types):
        if region_idx == -1 or region_idx >= len(vor.regions):
            continue
        region = vor.regions[region_idx]
        if -1 in region:
            continue
        polygon = [vor.vertices[i] for i in region]
        if polygon:
            ax.fill(*zip(*polygon), color=residue_color, alpha=0.6)

    voronoi_plot_2d(vor, ax=ax, show_vertices=False, line_colors="black")
    ax.scatter(points_2d[:, 0], points_2d[:, 1], c="black", s=0.1, label="Residues")

    plt.title(f"Amino Acid-Based Voronoi Diagram for {pdb_name}")
    plt.legend()

    output_path = os.path.join(output_folder, f"{pdb_name}.png")
    plt.savefig(output_path, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"Saved Voronoi diagram: {output_path}")


###############################################################################
# 2. FETCHING FUNCTIONS: PDBe (PDB) & AlphaFold
###############################################################################

def get_top_pdb_from_uniprot(uniprot_id):
    """
    Query the PDBe 'best_structures' API to get the top (first) PDB ID
    associated with a given UniProt ID.
    Returns the PDB ID string or None.
    """
    print(f"Fetching experimental PDB ID for UniProt ID: {uniprot_id}")
    url = f"https://www.ebi.ac.uk/pdbe/api/mappings/best_structures/{uniprot_id}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if uniprot_id in data and data[uniprot_id]:
            top_pdb_id = data[uniprot_id][0]['pdb_id']
            print(f"Selected PDB ID for {uniprot_id}: {top_pdb_id}")
            return top_pdb_id
    else:
        print(f"Error fetching data for {uniprot_id} (HTTP {response.status_code}).")

    print(f"No experimental PDB found for {uniprot_id}.")
    return None


def download_pdb(pdb_id, pdb_folder):
    """
    Download an experimental PDB ID from RCSB if not already present.
    Returns the local file path, or None if download fails.
    """
    if not pdb_id:
        return None

    pdb_file_path = os.path.join(pdb_folder, f"{pdb_id}.pdb")

    # Skip if already downloaded
    if os.path.exists(pdb_file_path):
        print(f"Skipping {pdb_id}: already downloaded.")
        return pdb_file_path

    print(f"Downloading PDB file: {pdb_id}.pdb")
    pdb_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(pdb_url)

    if response.status_code == 200:
        with open(pdb_file_path, "wb") as file:
            file.write(response.content)
        print(f"Successfully downloaded: {pdb_file_path}")
        return pdb_file_path
    else:
        print(f"Failed to download PDB {pdb_id} (HTTP {response.status_code})")
        return None


def download_alphafold(uniprot_id, pdb_folder):
    """
    Attempt to download the AlphaFold model for 'uniprot_id' (model_v4).
    The typical file name is: AF-<uniprot_id>-F1-model_v4.pdb
    Returns the local file path or None if not available.
    """
    af_id = f"AF-{uniprot_id}-F1"
    af_pdb_path = os.path.join(pdb_folder, f"{af_id}.pdb")
    if os.path.exists(af_pdb_path):
        print(f"AlphaFold model for {uniprot_id} already downloaded.")
        return af_pdb_path

    url = f"https://alphafold.ebi.ac.uk/files/{af_id}-model_v4.pdb"
    print(f"Trying AlphaFold model download: {url}")
    r = requests.get(url)
    if r.status_code == 200:
        with open(af_pdb_path, "wb") as f:
            f.write(r.content)
        print(f"Saved AlphaFold model to {af_pdb_path}")
        return af_pdb_path
    else:
        print(f"No AlphaFold model found for {uniprot_id} at {url}. (HTTP {r.status_code})")
        return None


def get_or_fetch_structure(uniprot_id, pdb_folder):
    """
    1) Tries to find an experimental PDB from PDBe.
    2) If none found or fails, tries downloading AlphaFold model.
    Returns (structure_id, source, local_path).
      - structure_id: e.g. "4XYZ" or "AF-Q9NRL2-F1"
      - source: "PDB" or "AlphaFold"
      - local_path: local .pdb file path
    If all fails, returns (None, None, None).
    """

    # 1) Attempt PDBe (experimental PDB)
    pdb_id = get_top_pdb_from_uniprot(uniprot_id)
    if pdb_id:
        local_path = download_pdb(pdb_id, pdb_folder)
        if local_path:
            return pdb_id, "PDB", local_path

    # 2) Attempt AlphaFold
    af_path = download_alphafold(uniprot_id, pdb_folder)
    if af_path:
        # e.g. "AF-Q9NRL2-F1" as the "structure ID"
        structure_id = os.path.splitext(os.path.basename(af_path))[0]  # "AF-Q9NRL2-F1"
        return structure_id, "AlphaFold", af_path

    # 3) If both fail
    return None, None, None


###############################################################################
# 3. MAIN WORKFLOW FUNCTION
###############################################################################

def process_folder(subfolder_path):
    """
    Given a folder path, look for an Excel file with the same name as the folder, then:
      - Load the Excel
      - For each row (uniprot_id):
          * Try PDBe => else AlphaFold => else mark unfetchable
          * Download structure, update DataFrame
      - Save updated Excel
      - Create Voronoi diagrams for any newly-downloaded structures
      - Log all failures in 'unfetchable_elements.txt'
    """

    folder_name = os.path.basename(subfolder_path)
    excel_file_name = f"{folder_name}.xlsx"
    excel_file_path = os.path.join(subfolder_path, excel_file_name)

    if not os.path.isfile(excel_file_path):
        print(f"  >> No matching Excel file ({excel_file_name}) found in '{folder_name}'. Skipping.\n")
        return

    excel_output_path = os.path.join(subfolder_path, f"{folder_name}_with_top_pdb.xlsx")
    unfetchable_file = os.path.join(subfolder_path, "unfetchable_elements.txt")

    print(f"\n=== Processing folder: {folder_name} ===")
    print(f"Loading Excel file: {excel_file_path}")
    df = pd.read_excel(excel_file_path)

    # Create or check folder for .pdb files
    pdb_folder = os.path.join(subfolder_path, "pdb_files")
    os.makedirs(pdb_folder, exist_ok=True)
    print(f"Created/checked folder for PDB files: {pdb_folder}")

    # Ensure columns exist
    if "top_pdb_id" not in df.columns:
        df["top_pdb_id"] = None

    # Optional new column to show if structure is from PDB or AlphaFold
    if "structure_source" not in df.columns:
        df["structure_source"] = None

    if "pdb_file_path" not in df.columns:
        df["pdb_file_path"] = None

    # ----------------------------------------------------------------
    # Fetch or generate structure for each row
    # ----------------------------------------------------------------
    for index, row in df.iterrows():
        uniprot_id = row.get("uniprot_id", None)
        if not uniprot_id or pd.isna(uniprot_id):
            print(f"Row {index} has no 'uniprot_id'. Skipping.")
            continue

        # If we already have a file path, skip
        existing_path = row.get("pdb_file_path", None)
        if pd.notna(existing_path):
            print(f"Skipping UniProt {uniprot_id}: structure already downloaded.")
            continue

        # Attempt PDBe => If no, try AlphaFold
        structure_id, source, local_path = get_or_fetch_structure(uniprot_id, pdb_folder)

        if not structure_id or not local_path:
            # Mark as unfetchable
            msg = f"No structure found for UniProt {uniprot_id}\n"
            print(msg)
            with open(unfetchable_file, "a") as f:
                f.write(msg)

            df.to_excel(excel_output_path, index=False)
            continue

        # Update DF
        df.at[index, "top_pdb_id"] = structure_id
        df.at[index, "structure_source"] = source
        df.at[index, "pdb_file_path"] = local_path

        # Save progress
        df.to_excel(excel_output_path, index=False)

    print(f"Finished structure retrieval. Updated Excel: {excel_output_path}")

    # ----------------------------------------------------------------
    # Generate Voronoi Diagrams
    # ----------------------------------------------------------------
    print(f"Generating Voronoi diagrams for: {folder_name}")
    log_file = os.path.join(subfolder_path, "processed_files.txt")

    if os.path.exists(log_file):
        with open(log_file, "r") as f:
            processed_files = set(f.read().splitlines())
    else:
        processed_files = set()

    # Reload updated DataFrame
    df = pd.read_excel(excel_output_path)

    voronoi_folder = os.path.join(subfolder_path, "voronoi_images")
    os.makedirs(voronoi_folder, exist_ok=True)

    for index, row in df.iterrows():
        pdb_path = row.get("pdb_file_path", None)
        if not pdb_path or pd.isna(pdb_path):
            continue

        pdb_file_name = os.path.basename(pdb_path)
        if pdb_file_name in processed_files:
            print(f"Skipping {pdb_file_name}: Already processed for Voronoi.")
            continue

        if not os.path.exists(pdb_path):
            print(f"Warning: PDB file not found: {pdb_path}. Skipping.")
            continue

        print(f"Processing Voronoi for {pdb_file_name}...")
        residues, residue_colors = extract_residue_coordinates_and_types(pdb_path)
        if len(residues) == 0:
            print(f"Warning: No residues found in {pdb_file_name}.")
            continue

        plot_voronoi_2d_residue_colored(residues, residue_colors, pdb_file_name, voronoi_folder)
        with open(log_file, "a") as f:
            f.write(pdb_file_name + "\n")

    print(f"Completed Voronoi diagrams for folder: {folder_name}\n")


###############################################################################
# 4. MAIN SCRIPT — ONLY ONE FOLDER
###############################################################################

if __name__ == "__main__":
    # Adjust folder_to_process to match your folder & Excel name
    folder_to_process = "tumor_antigens_non-immunogenic"
    project_folder = os.getcwd()
    full_subfolder_path = os.path.join(project_folder, folder_to_process)

    process_folder(full_subfolder_path)
    print("\nAll done!")



=== Processing folder: protein_antigens_non-immunogenic ===
Loading Excel file: /Users/marcobenavides/Documents/Columbia University/Spring 2025/DL Biomedical Imaging/Project/protein_antigens_non-immunogenic/protein_antigens_non-immunogenic.xlsx
Created/checked folder for PDB files: /Users/marcobenavides/Documents/Columbia University/Spring 2025/DL Biomedical Imaging/Project/protein_antigens_non-immunogenic/pdb_files
Fetching experimental PDB ID for UniProt ID: B0QZ99
Error fetching data for B0QZ99 (HTTP 404).
No experimental PDB found for B0QZ99.
Trying AlphaFold model download: https://alphafold.ebi.ac.uk/files/AF-B0QZ99-F1-model_v4.pdb
Saved AlphaFold model to /Users/marcobenavides/Documents/Columbia University/Spring 2025/DL Biomedical Imaging/Project/protein_antigens_non-immunogenic/pdb_files/AF-B0QZ99-F1.pdb
Fetching experimental PDB ID for UniProt ID: P62829
Selected PDB ID for P62829: 8a3d
Failed to download PDB 8a3d (HTTP 404)
Trying AlphaFold model download: https://alphafold

For Bacterial Fasta files

In [60]:
import os
import requests
from Bio import SeqIO, PDB
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d

###############################################################################
# 1. HELPER FUNCTIONS
###############################################################################

def extract_uniprot_id(header):
    """
    For headers like 'sp|Q9NRL2|PROTEIN_NAME ...', returns 'Q9NRL2'.
    Modify if your FASTA headers differ.
    """
    parts = header.split("|")
    if len(parts) >= 2:
        return parts[1]
    return None

def blast_or_map_fasta_sequence(seq):
    """
    Placeholder if the header doesn't contain a UniProt ID. 
    Here you'd do a BLAST or other lookup to get a UniProt ID for 'seq'.
    """
    return None

def get_top_pdb_from_uniprot(uniprot_id):
    """
    Query PDBe 'best_structures' to see if there's an experimental PDB for 'uniprot_id'.
    Returns the top PDB ID (string) or None if no structure is found.
    """
    print(f"Fetching PDB ID for UniProt ID: {uniprot_id}")
    url = f"https://www.ebi.ac.uk/pdbe/api/mappings/best_structures/{uniprot_id}"
    resp = requests.get(url)
    if resp.status_code == 200:
        data = resp.json()
        if uniprot_id in data and data[uniprot_id]:
            top_pdb_id = data[uniprot_id][0]['pdb_id']
            print(f"Selected PDB ID for {uniprot_id}: {top_pdb_id}")
            return top_pdb_id
    print(f"No experimental PDB for {uniprot_id}")
    return None

def download_pdb_from_rcsb(pdb_id, pdb_folder):
    """
    Downloads an experimental PDB ID from RCSB if not already present.
    Returns the local file path or None if fail.
    """
    pdb_path = os.path.join(pdb_folder, f"{pdb_id}.pdb")
    if os.path.exists(pdb_path):
        print(f"{pdb_id} already downloaded (RCSB).")
        return pdb_path

    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    print(f"Downloading {pdb_id} from {url} ...")
    r = requests.get(url)
    if r.status_code == 200:
        with open(pdb_path, "wb") as f:
            f.write(r.content)
        print(f"Saved experimental PDB to {pdb_path}")
        return pdb_path

    print(f"Failed to download PDB {pdb_id} from RCSB")
    return None

def download_alphafold_model(uniprot_id, pdb_folder):
    """
    Attempts to download an AlphaFold model (.pdb) from alphafold.ebi.ac.uk.
    The typical file path is: 
       https://alphafold.ebi.ac.uk/files/AF-<UNIPROT_ID>-F1-model_v4.pdb
    or model_v3, etc. We'll try model_v4.
    Returns the local file path if successful, else None.
    """
    # Construct a "synthetic" ID like 'AF-Q9NRL2-F1' for logging
    alpha_id = f"AF-{uniprot_id}-F1"
    pdb_path = os.path.join(pdb_folder, f"{alpha_id}.pdb")
    if os.path.exists(pdb_path):
        print(f"{alpha_id} already downloaded (AlphaFold DB).")
        return pdb_path

    url = f"https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v4.pdb"
    print(f"Attempting AlphaFold model download from {url} ...")
    r = requests.get(url)
    if r.status_code == 200:
        with open(pdb_path, "wb") as f:
            f.write(r.content)
        print(f"Saved AlphaFold model to {pdb_path}")
        return pdb_path

    print(f"No AlphaFold model found for {uniprot_id} at {url}")
    return None

def run_local_alphafold(uniprot_id, sequence, pdb_folder):
    """
    Placeholder for generating an AlphaFold prediction locally.
    This is non-trivial; typically you'd run a Docker image or a local 
    AlphaFold installation with the full DBs. 
    Return the path to the newly generated PDB or None if it fails.
    """
    alpha_id = f"AF_LOCAL-{uniprot_id}-F1"
    pdb_path = os.path.join(pdb_folder, f"{alpha_id}.pdb")

    # Pseudocode:
    """
    cmd = f"./run_alphafold.sh --uniprot {uniprot_id} --sequence '{sequence}' --output {pdb_path}"
    ret = os.system(cmd)
    if ret == 0 and os.path.exists(pdb_path):
        return pdb_path
    """
    print(f"[PLACEHOLDER] Generating local AlphaFold structure for {uniprot_id} ...")
    print(f"[PLACEHOLDER] Put your local AlphaFold invocation here.")
    return None  # Return None to indicate we haven't actually done it.

def get_or_generate_structure(uniprot_id, sequence, pdb_folder):
    """
    Unified function that:
      1) Tries to get an experimental PDB from PDBe.
      2) If none, tries AlphaFold DB.
      3) If none, tries local AlphaFold generation.
    Returns (local_pdb_path, structure_label) or (None, None) if fails.
    
    'structure_label' might be:
       "PDB:<pdb_id>" or "AF_REMOTE:AF-<uniprot>-F1" or "AF_LOCAL:AF_LOCAL-<uniprot>-F1"
    """
    # 1) Attempt top experimental PDB
    pdb_id = get_top_pdb_from_uniprot(uniprot_id)
    if pdb_id:
        local_path = download_pdb_from_rcsb(pdb_id, pdb_folder)
        if local_path:
            return local_path, f"PDB:{pdb_id}"

    # 2) Attempt AlphaFold DB
    alpha_path = download_alphafold_model(uniprot_id, pdb_folder)
    if alpha_path:
        # e.g. alpha_id = "AF-Q9NRL2-F1"
        alpha_id = os.path.splitext(os.path.basename(alpha_path))[0]  # "AF-Q9NRL2-F1-model_v4"
        return alpha_path, f"AF_REMOTE:{alpha_id}"

    # 3) Attempt local AlphaFold generation
    local_path = run_local_alphafold(uniprot_id, sequence, pdb_folder)
    if local_path:
        local_id = os.path.splitext(os.path.basename(local_path))[0]
        return local_path, f"AF_LOCAL:{local_id}"

    # If all fail
    return None, None

###############################################################################
# 2. PLOTTING & ANALYSIS
###############################################################################

amino_acid_colors = {
    "ALA": "#FF0000",  "ARG": "#00FF00", "ASN": "#0000FF", "ASP": "#FF00FF",
    "CYS": "#FFFF00",  "GLN": "#FF8000", "GLU": "#8000FF", "GLY": "#808080",
    "HIS": "#008000",  "ILE": "#00FFFF", "LEU": "#0040FF", "LYS": "#800000",
    "MET": "#008080",  "PHE": "#FF8080", "PRO": "#800080", "SER": "#40E0D0",
    "THR": "#A52A2A",  "TRP": "#FFD700", "TYR": "#DC143C", "VAL": "#006400"
}
default_color = "#000000"

def extract_residues_and_colors(pdb_file):
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure("prot", pdb_file)

    # Initialize empty lists directly
    coords = []
    colors = []

    for model in structure:
        for chain in model:
            for residue in chain:
                if "CA" in residue:
                    coords.append(residue["CA"].coord)
                    rname = residue.get_resname().strip()
                    colors.append(amino_acid_colors.get(rname, default_color))

    return np.array(coords), colors


def plot_voronoi(coords, colors, structure_label, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    if len(coords) < 2:
        print(f"Not enough points for Voronoi in {structure_label}.")
        return

    points_2d = coords[:, :2]
    vor = Voronoi(points_2d)

    fig, ax = plt.subplots(figsize=(8, 8))
    for region_idx, color_val in zip(vor.point_region, colors):
        if region_idx == -1 or region_idx >= len(vor.regions):
            continue
        region = vor.regions[region_idx]
        if -1 in region:
            continue
        polygon = [vor.vertices[i] for i in region]
        if polygon:
            ax.fill(*zip(*polygon), alpha=0.6, color=color_val)

    voronoi_plot_2d(vor, ax=ax, show_vertices=False, line_colors="black")
    ax.scatter(points_2d[:, 0], points_2d[:, 1], c="black", s=0.1)
    plt.title(f"Voronoi for {structure_label}")
    plt.tight_layout()

    # For the PNG filename, let's remove special chars:
    safe_label = structure_label.replace(":", "_").replace("/", "_")
    png_path = os.path.join(output_folder, f"{safe_label}.png")
    plt.savefig(png_path, dpi=300)
    plt.close()
    print(f"Saved Voronoi: {png_path}")

###############################################################################
# 3. MAIN PIPELINE
###############################################################################

def process_fasta(fasta_path, output_folder="output_for_fasta"):
    """
    Reads each record from the given FASTA file,
    tries to get PDB or AlphaFold structure,
    logs any failures,
    generates Voronoi diagrams.
    """
    os.makedirs(output_folder, exist_ok=True)
    pdb_folder = os.path.join(output_folder, "pdb_files")
    os.makedirs(pdb_folder, exist_ok=True)
    voronoi_folder = os.path.join(output_folder, "voronoi_images")
    os.makedirs(voronoi_folder, exist_ok=True)

    # Log files
    unfetchable_path = os.path.join(output_folder, "unfetchable_elements.txt")
    processed_path = os.path.join(output_folder, "processed_files.txt")

    if os.path.exists(processed_path):
        with open(processed_path, "r") as f:
            processed_files = set(f.read().splitlines())
    else:
        processed_files = set()

    # Read the FASTA
    for record in SeqIO.parse(fasta_path, "fasta"):
        header = record.description
        sequence = str(record.seq)
        print(f"\nFASTA entry: {header}")

        # 1) UniProt or BLAST
        uniprot_id = extract_uniprot_id(header)
        if not uniprot_id:
            uniprot_id = blast_or_map_fasta_sequence(sequence)
            if not uniprot_id:
                print(f"No UniProt ID for {header}; skipping.")
                with open(unfetchable_path, "a") as uf:
                    uf.write(f"Could NOT map header [{header}] to UniProt.\n")
                continue

        # 2) Attempt to get or generate a structure
        local_pdb, label = get_or_generate_structure(uniprot_id, sequence, pdb_folder)
        if not local_pdb or not label:
            print(f"All attempts failed for {uniprot_id}. Logging in unfetchable.")
            with open(unfetchable_path, "a") as uf:
                uf.write(f"No structure found for UniProt {uniprot_id}\n")
            continue

        # 3) Check if we've processed Voronoi for that local PDB
        pdb_filename = os.path.basename(local_pdb)  # e.g. "PDBID.pdb" or "AF-XYZ.pdb"
        if pdb_filename in processed_files:
            print(f"Already made Voronoi for {pdb_filename}. Skipping.")
            continue

        # 4) Extract coords & colors
        coords, residue_colors = extract_residues_and_colors(local_pdb)
        if len(coords) == 0:
            print(f"No alpha carbons in {pdb_filename}. Logging.")
            with open(unfetchable_path, "a") as uf:
                uf.write(f"No residues in {pdb_filename} for {uniprot_id}\n")
            continue

        # 5) Plot Voronoi
        plot_voronoi(coords, residue_colors, label, voronoi_folder)

        # Mark as processed
        with open(processed_path, "a") as pf:
            pf.write(pdb_filename + "\n")

    print("\nDone processing all FASTA entries!")

###############################################################################
# 4. MAIN
###############################################################################

if __name__ == "__main__":
    fasta_file = "bacterial_dataset.fasta"  # or your .fasta
    process_fasta(fasta_file, output_folder="bacterial_dataset_antigens")



FASTA entry: sp|P08191|FIMH_ECOLI Type 1 fimbrin D-mannose specific adhesin OS=Escherichia coli (strain K12) OX=83333 GN=fimH PE=1 SV=2
Fetching PDB ID for UniProt ID: P08191
Selected PDB ID for P08191: 4xo9
4xo9 already downloaded (RCSB).
Already made Voronoi for 4xo9.pdb. Skipping.

FASTA entry: sp|P17315|CIRA_ECOLI Colicin I receptor OS=Escherichia coli (strain K12) OX=83333 GN=cirA PE=1 SV=2
Fetching PDB ID for UniProt ID: P17315
Selected PDB ID for P17315: 2hdi
2hdi already downloaded (RCSB).
Already made Voronoi for 2hdi.pdb. Skipping.

FASTA entry: tr|Q7DHH4|Q7DHH4_STAAU MecA OS=Staphylococcus aureus OX=1280 GN=mecA PE=1 SV=1
Fetching PDB ID for UniProt ID: Q7DHH4
Selected PDB ID for Q7DHH4: 5m18
5m18 already downloaded (RCSB).
Already made Voronoi for 5m18.pdb. Skipping.

FASTA entry: tr|Q9RM68|Q9RM68_CLOPF Epsilon toxin (Fragment) OS=Clostridium perfringens D OX=107819 GN=etx PE=4 SV=1
Fetching PDB ID for UniProt ID: Q9RM68
No experimental PDB for Q9RM68
Attempting AlphaFold 

Generate the graphs for the Graph Neural Network

In [8]:
import os
import torch
from torch_geometric.data import Data
from Bio import PDB
import numpy as np

# Helper function to extract amino acid residues from PDB
def extract_residues_and_coordinates(pdb_file):
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure("prot", pdb_file)

    residues = []
    coordinates = []

    # Loop through models, chains, and residues
    for model in structure:
        for chain in model:
            for residue in chain:
                if "CA" in residue:  # Only use C-alpha atoms
                    residues.append(residue)
                    coordinates.append(residue["CA"].coord)  # Coordinates of C-alpha atoms

    return residues, np.array(coordinates)

# Generate a graph from the PDB structure
def generate_graph_from_pdb(pdb_file, threshold=5.0):
    residues, coordinates = extract_residues_and_coordinates(pdb_file)
    num_residues = len(residues)

    # Nodes: Each residue is a node
    x = torch.tensor(coordinates, dtype=torch.float)  # Residue coordinates as node features

    # Edges: Create edges based on distance threshold between residues
    edge_index = []
    for i in range(num_residues):
        for j in range(i + 1, num_residues):
            distance = np.linalg.norm(coordinates[i] - coordinates[j])  # Calculate Euclidean distance
            if distance < threshold:  # Only create edges if distance is below threshold
                edge_index.append([i, j])  # Undirected edges: (i, j) and (j, i)
                edge_index.append([j, i])

    # Convert edge_index to tensor
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    # Return Data object with graph information
    return Data(x=x, edge_index=edge_index)

# Save graph data
def save_graph_data(pdb_file, graph_dir):
    os.makedirs(graph_dir, exist_ok=True)
    graph = generate_graph_from_pdb(pdb_file)
    pdb_id = os.path.splitext(os.path.basename(pdb_file))[0]  # Get PDB ID without extension
    graph_path = os.path.join(graph_dir, f"{pdb_id}_graph.pth")
    torch.save(graph, graph_path)
    print(f"Graph for {pdb_id} saved to {graph_path}")

# Main function to process folders
def process_folders(base_dir):
    # Loop over all folders in the base directory
    for folder_name in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder_name)

        # Only process directories
        if os.path.isdir(folder_path):
            pdb_folder = os.path.join(folder_path, "pdb_files")
            graph_dir = os.path.join(folder_path, "graphs")

            # Check if both 'pdb_files' and 'graphs' directories exist
            if os.path.exists(pdb_folder) and os.path.exists(graph_dir):
                print(f"Processing folder: {folder_name}")

                # Loop over all PDB files in the 'pdb_files' folder
                pdb_files = [f for f in os.listdir(pdb_folder) if f.endswith(".pdb")]
                for pdb_file in pdb_files:
                    pdb_path = os.path.join(pdb_folder, pdb_file)
                    pdb_id = os.path.splitext(pdb_file)[0]
                    graph_path = os.path.join(graph_dir, f"{pdb_id}_graph.pth")

                    # Check if the corresponding graph already exists
                    if not os.path.exists(graph_path):
                        print(f"Generating graph for {pdb_id}...")
                        save_graph_data(pdb_path, graph_dir)
                    else:
                        print(f"Graph for {pdb_id} already exists. Skipping.")
            else:
                print(f"Skipping {folder_name} as it doesn't contain 'pdb_files' or 'graphs' directories.")

# Example usage
base_dir = "."  # Current directory
process_folders(base_dir)


Skipping Writings as it doesn't contain 'pdb_files' or 'graphs' directories.
Processing folder: protein_antigens_immunogenic
Generating graph for AF-O15213-F1...
Graph for AF-O15213-F1 saved to ./protein_antigens_immunogenic/graphs/AF-O15213-F1_graph.pth
Generating graph for 4pl8...
Graph for 4pl8 saved to ./protein_antigens_immunogenic/graphs/4pl8_graph.pth
Generating graph for AF-Q13797-F1...
Graph for AF-Q13797-F1 saved to ./protein_antigens_immunogenic/graphs/AF-Q13797-F1_graph.pth
Generating graph for 2j51...
Graph for 2j51 saved to ./protein_antigens_immunogenic/graphs/2j51_graph.pth
Generating graph for AF-Q9UJ68-F1...
Graph for AF-Q9UJ68-F1 saved to ./protein_antigens_immunogenic/graphs/AF-Q9UJ68-F1_graph.pth
Generating graph for 7y68...
Graph for 7y68 saved to ./protein_antigens_immunogenic/graphs/7y68_graph.pth
Generating graph for 8cbl...
Graph for 8cbl saved to ./protein_antigens_immunogenic/graphs/8cbl_graph.pth
Generating graph for 9bz4...
Graph for 9bz4 saved to ./protei

Code to visualize the graphs generated

In [ ]:
import os
import torch
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.data import Data

# Function to load and visualize the graph from a PDB file
def visualize_graph_from_pdb(pdb_file, graph_dir):
    # Extract PDB ID (without extension)
    pdb_id = os.path.splitext(os.path.basename(pdb_file))[0]
    graph_path = os.path.join(graph_dir, f"{pdb_id}_graph.pth")
    
    # Load the graph with safe_globals to allow deserialization of PyTorch Geometric classes
    with torch.serialization.safe_globals([Data]):
        try:
            graph = torch.load(graph_path)
            print(f"Graph loaded from {graph_path}")
        except Exception as e:
            print(f"Failed to load the graph: {e}")
            return
    
    # Create a NetworkX graph from PyTorch Geometric Data object
    G = nx.Graph()
    
    # Add nodes with positions
    for i in range(graph.num_nodes):
        G.add_node(i, pos=graph.x[i].numpy())  # Node position is from the coordinates
    
    # Add edges
    for i, j in graph.edge_index.t().numpy():
        G.add_edge(i, j)
    
    # Get positions from node features (coordinates of residues)
    pos = nx.get_node_attributes(G, 'pos')

    # Plot the graph
    plt.figure(figsize=(8, 8))
    nx.draw(G, pos, with_labels=False, node_size=50, node_color="blue", alpha=0.6)
    plt.title(f"Graph Visualization for {pdb_id}")
    plt.tight_layout()
    plt.show()

# Example usage:
pdb_file = "tumor_antigens_non-immunogenic/pdb_files/1bnd.pdb"  # Replace with your actual file path
graph_dir = "tumor_antigens_non-immunogenic/graphs/"  # Directory where graph data is stored
visualize_graph_from_pdb(pdb_file, graph_dir)


Failed to load the graph: Weights only load failed. This file can still be loaded, to do so you have two options, do those steps only if you trust the source of the checkpoint. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL torch_geometric.data.data.DataEdgeAttr was not an allowed global by default. Please use `torch.serialization.add_safe_globals([DataEdgeAttr])` or the `torch.serialization.safe_globals([DataEdgeAttr])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepte